In [2]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import adam_v2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [30]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

## Setting up the image augmentation from last Lesson: 
Note that this section of the code has been pre-written for you and does not need to be changed, just run. If you would like to change the ImageDataGenerator parameters, feel free.

In [4]:
## This is the image size that VGG16 takes as input
IMG_SIZE = (224, 224)

In [5]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [31]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [32]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [33]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [34]:
vgg_model.layers

In [35]:
## Now, choose which layers of VGG16 we actually want to fine-tune
## Here, we'll freeze all but the last convolutional layer

## Add some code here to freeze all but the last convolutional layer:
##### Your code here ######

for layer in vgg_model.layers[0:17]:
    layer.trainable = False
    

In [36]:
## Check to make sure you froze the right ones: 
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


## Build a simple sequential model using only the VGG16 architecture
Note the code in the cell below has been pre-written for you, you only need to run it

In [37]:
## Build your model using the mostly-frozen VGG16 architecture: 
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='relu'))

In [38]:
## Set our optimizer, loss function, and learning rate (you can change the learning rate here if you'd like)
## but otherwise this cell can be run as is
optimizer = adam_v2.Adam(learning_rate=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [39]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [40]:
## Just run a single epoch to see how it does:
new_model.fit(train_gen, validation_data = (testX, testY), epochs = 5)

Epoch 1/5
3/3 [==============================] - 8s 2s/step - loss: 1.9492 - binary_accuracy: 0.5508 - val_loss: 5.7601 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 6s 2s/step - loss: 3.8577 - binary_accuracy: 0.4083 - val_loss: 0.7194 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 6s 2s/step - loss: 0.6310 - binary_accuracy: 0.6056 - val_loss: 0.5962 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 6s 2s/step - loss: 0.2359 - binary_accuracy: 0.8694 - val_loss: 0.5892 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 6s 2s/step - loss: 0.4831 - binary_accuracy: 0.7222 - val_loss: 2.8453 - val_binary_accuracy: 0.6667


## Let's try another experiment where we add a few more dense layers:

In [41]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a couple of dense (aka. fully-connected) layers.
# This is for combining features that the VGG16 model has
# recognized in the image.

##### Your code here ######
new_model.add(Dense(1, activation='relu'))
new_model.add(Dense(1, activation='relu'))
new_model.add(Dense(1, activation='relu'))

# Final output layer:
new_model.add(Dense(1, activation='relu'))

In [42]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [43]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 8s 2s/step - loss: 8.1020 - binary_accuracy: 0.4747 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 5s 2s/step - loss: 8.8810 - binary_accuracy: 0.4242 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 6s 2s/step - loss: 8.1409 - binary_accuracy: 0.4722 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 10s 3s/step - loss: 8.1020 - binary_accuracy: 0.4747 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 7s 2s/step - loss: 7.9267 - binary_accuracy: 0.4861 - val_loss: 7.7125 - val_binary_accuracy: 0.5000


## Now let's add dropout and another fully connected layer:

In [64]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add several fully-connected layers with dropout
##### Your code here ######
new_model.add(Dropout(0.5))
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(256, activation='relu'))
# new_model.add(Dropout(0.5))
# new_model.add(Dense(64, activation='relu'))
# new_model.add(Dropout(0.5))
# new_model.add(Dense(1, activation='relu'))
# new_model.add(Dropout(0.2))
# new_model.add(Dense(1, activation='relu'))
# tf.keras.layers.Dropout(.2,



# Final output layer
new_model.add(Dense(1, activation='sigmoid'))

In [65]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [66]:
## Just run a single epoch to see how it does:
new_model.fit(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 10s 3s/step - loss: 0.9111 - binary_accuracy: 0.4864 - val_loss: 0.6699 - val_binary_accuracy: 0.6667
Epoch 2/5
3/3 [==============================] - 6s 3s/step - loss: 0.6052 - binary_accuracy: 0.7568 - val_loss: 0.7219 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 6s 2s/step - loss: 1.2133 - binary_accuracy: 0.3015 - val_loss: 0.9153 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 6s 2s/step - loss: 0.5313 - binary_accuracy: 0.6972 - val_loss: 0.7090 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 7s 2s/step - loss: 1.2602 - binary_accuracy: 0.6003 - val_loss: 1.4885 - val_binary_accuracy: 0.5000
